<a href="https://colab.research.google.com/github/Saumitra-Shukla/Low-resolution-to-High-resolution-converter-GAN/blob/master/dataset_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [30]:
!ls

DIV2K_train_HR		DIV2K_valid_HR		drive
DIV2K_train_LR_bicubic	DIV2K_valid_LR_bicubic


In [0]:
! rm -rf x.pickle

In [0]:
!ls "/content/drive/My Drive/machine-learning/dataset"

In [5]:
import os,subprocess
path="/content/drive/My Drive/machine-learning/dataset"
for i in os.listdir(path):
  nn=os.path.join(path,i)
  print(nn)
  subprocess.call(['cp' ,nn ,i])
  subprocess.call(['unzip' ,i])
  subprocess.call(['rm' ,'-rf' ,i])

/content/drive/My Drive/machine-learning/dataset/DIV2K_train_HR.zip
/content/drive/My Drive/machine-learning/dataset/DIV2K_valid_HR.zip
/content/drive/My Drive/machine-learning/dataset/DIV2K_train_LR_bicubic_X4.zip
/content/drive/My Drive/machine-learning/dataset/DIV2K_valid_LR_bicubic_X4.zip


In [0]:
!ls DIV2K_train_LR_bicubic/X4

In [0]:
!pip install tf-nightly-gpu-2.0-preview

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from keras.preprocessing.image import ImageDataGenerator as idg

Using TensorFlow backend.


In [0]:
gen=idg(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1,
                                              shear_range=0.1, zoom_range=0.1,
                                             channel_shift_range=10, horizontal_flip=True, vertical_flip=True)

In [0]:
def plots(ims,figsize=(12,6),rows=1,interp=False,titles=None):
  if type(ims[0]) is np.ndarray:
    ims=np.array(ims).astype(np.uint8)
    if (ims.shape[-1]!=3):
      ims=ims.transpose((0,2,3,1))
  f=plt.figure(figsize=figsize)
  cols=len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows +1
  for i in range(len(ims)):
    sp=f.add_subplot(rows,cols,i+1)
    sp.axis('Off')
    if titles is not None:
      sp.set_title(titles[i],fontsize=16)
    plt.imshow(ims[i], interpolation=None if interp else 'none')

In [0]:
categ=['HR','LR_bicubic']
fl_str='DIV2K_train_'
for c in categ:
  fl_path=fl_str+c
  for img in os.listdir(fl_path):
    try:
      img_path=os.path.join(fl_path,img)
      img_array=cv2.imread(img_path)
      plt.imshow(img_array)
      plt.show()
      aug_iter=gen.flow(np.expand_dims(img_array,0))
      for i in range(10):
      #aug_img=next(aug_iter)
        plots(next(aug_iter)[0].astype(np.uint8),figsize=(28,18),rows=2)
        plt.show()
    except Exception as e:
      continue

    break 
  break

In [0]:
categ=['HR','LR_bicubic/X4']
training_data=[]
img_size=1500
fl_str='DIV2K_train_'
for c in categ:
  fl_path=fl_str+c
  i=categ.index(c)
  for img in os.listdir(fl_path):
    try:
      img_path=os.path.join(fl_path,img)
      img_array=cv2.imread(img_path)
      img_array=cv2.resize(img_array,(img_size,img_size))
      #aug_iter=gen.flow(np.expand_dims(img_array,0))
      training_data.append([img_array,i])
      '''
      for i in range(10):
      #aug_img=next(aug_iter)
        #.astype(np.uint8)
        training_data.append([next(aug_iter)[0],i])
        
      #training_data.append([img_array,i])
      '''
    except Exception as e:
      pass

In [0]:
img=cv2.imread("/content/drive/My Drive/machine-learning/dataset/DIV2K_train_LR_bicubic/")

In [6]:
print(len(training_data))

800


In [0]:
x=[]
y=[]
for features,label in training_data:
    x.append(features)
    y.append(label)
    
#x=np.array(x).reshape(-1,100,100,1)



In [4]:
print(len(y))

1600


In [0]:

import hickle
data=[x,y]
hickle.dump( data, 'data_file.hkl' )

In [20]:
!ls

data_file.hkl  DIV2K_train_HR	       DIV2K_valid_HR	       drive
data.json      DIV2K_train_LR_bicubic  DIV2K_valid_LR_bicubic


In [0]:
!cp data_file.hkl 'drive/My Drive/machine-learning/data_train.hkl'